In [1]:
!pip install pyreadstat

In [2]:
import pandas as pd
import pyreadstat
import os

# === 1. Find all XPT files in the same folder ===
DATA_DIR = os.getcwd()
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".xpt")]

print("Found XPT files:")
for f in files:
    print(" -", f)

print("\n=====================================\n")

# === 2. Safe loader (works on all pyreadstat versions) ===
def load_xpt(file):
    try:
        df, meta = pyreadstat.read_xport(
            file,
            encoding="latin1",   # safest for NHANES
        )
        print(f"[OK] Loaded {file} → {df.shape[0]} rows, {df.shape[1]} columns")
        return df
    except Exception as e:
        print(f"[FAILED] {file} → {e}")
        return None


# === 3. Load all XPT files into a dictionary ===
dfs = {}
for f in files:
    key = f.replace(".xpt", "").replace(".XPT", "")
    df = load_xpt(f)
    if df is not None:
        dfs[key] = df

print("\nLoaded datasets:", list(dfs.keys()))


# === 4. Find DEMO file (base for merge) ===
demo_key = None
for k in dfs.keys():
    if k.lower().startswith("demo"):  
        demo_key = k
        break

if demo_key is None:
    raise Exception("No DEMO file found. NHANES cannot be merged without DEMO_xxx.XPT")

print(f"\nUsing '{demo_key}' as base dataset.\n")

merged = dfs[demo_key].copy()


# === 5. Merge all other tables by SEQN ===
for key, df in dfs.items():
    if key == demo_key:
        continue

    # Avoid duplicated columns except SEQN
    df = df.loc[:, ~df.columns.duplicated()]

    old_cols = set(merged.columns)
    new_cols = set(df.columns) - {"SEQN"}

    print(f"Merging {key} → adding {len(new_cols)} new columns")

    merged = merged.merge(df, on="SEQN", how="left")


# === 6. Save final merged dataset ===
merged.to_csv("NHANES_MERGED.csv", index=False)
print("\n=====================================")
print("Final merged dataset saved as NHANES_MERGED.csv")
print("Rows:", merged.shape[0], " Columns:", merged.shape[1])


Found XPT files:
 - ALB_CR_L.xpt
 - BAX_L.xpt
 - BMX_L.xpt
 - BPQ_L.xpt
 - BPXO_L.xpt
 - DEMO_L.xpt
 - DIQ_L.xpt
 - GHB_L.xpt
 - GLU_L.xpt
 - HDL_L.xpt
 - INS_L.xpt
 - PAQ_L.xpt
 - SMQ_L.xpt
 - TRIGLY_L.xpt


[OK] Loaded ALB_CR_L.xpt → 8493 rows, 8 columns
[OK] Loaded BAX_L.xpt → 4771 rows, 45 columns
[OK] Loaded BMX_L.xpt → 8860 rows, 22 columns
[OK] Loaded BPQ_L.xpt → 8501 rows, 6 columns
[OK] Loaded BPXO_L.xpt → 7801 rows, 12 columns
[OK] Loaded DEMO_L.xpt → 11933 rows, 27 columns
[OK] Loaded DIQ_L.xpt → 11744 rows, 9 columns
[OK] Loaded GHB_L.xpt → 7199 rows, 3 columns
[OK] Loaded GLU_L.xpt → 3996 rows, 4 columns
[OK] Loaded HDL_L.xpt → 8068 rows, 4 columns
[OK] Loaded INS_L.xpt → 3996 rows, 5 columns
[OK] Loaded PAQ_L.xpt → 8153 rows, 8 columns
[OK] Loaded SMQ_L.xpt → 9015 rows, 9 columns
[OK] Loaded TRIGLY_L.xpt → 3996 rows, 10 columns

Loaded datasets: ['ALB_CR_L', 'BAX_L', 'BMX_L', 'BPQ_L', 'BPXO_L', 'DEMO_L', 'DIQ_L', 'GHB_L', 'GLU_L', 'HDL_L', 'INS_L', 'PAQ_L', 'SMQ_L', 'TRIG

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("NHANES_MERGED.csv")

# ====================================================================
# Helper function: choose first available variable from multiple names
# ====================================================================
def pick(df, names):
    for n in names:
        if n in df.columns:
            return n
    return None


# ====================================================================
# 1. Define variables to keep (with comments)
# ====================================================================

keep = {}

# Demographics
keep["SEQN"] = "Participant ID"
keep[pick(df, ["RIDAGEYR"])] = "Age (years)"
keep[pick(df, ["RIAGENDR"])] = "Gender (1=Male,2=Female)"
keep[pick(df, ["RIDRETH1"])] = "Race/Ethnicity"
keep[pick(df, ["INDFMPIR"])] = "Income-to-poverty ratio"

# Diabetes labs
keep[pick(df, ["LBXGH"])] = "HbA1c (%)"
keep[pick(df, ["LBDGLUSI", "LBXGLU"])] = "Fasting plasma glucose (mg/dL)"
keep[pick(df, ["LBDLDL"])] = "LDL cholesterol (mg/dL)"
keep[pick(df, ["URXUMA"])] = "Urine albumin (mg/L)"

# Anthropometrics
keep[pick(df, ["BMXBMI"])] = "BMI (kg/m2)"
keep[pick(df, ["BMXWAIST"])] = "Waist circumference (cm)"

# Diabetes questionnaire
keep[pick(df, ["DIQ010"])] = "Doctor told you have diabetes"
keep[pick(df, ["DIQ050"])] = "Taking insulin now"

# ====================================================================
# 2. Blood Pressure (BPXO) Integration
# ====================================================================

# Raw systolic and diastolic readings
bp_vars = [
    "BPXOSY1","BPXOSY2","BPXOSY3",   # Systolic
    "BPXODI1","BPXODI2","BPXODI3",   # Diastolic
    "BPXOPLS1","BPXOPLS2","BPXOPLS3",# Pulse rate
    "BPAOARM",                       # Arm used
    "BPAOCSZ"                        # Cuff size
]

for v in bp_vars:
    if v in df.columns:
        keep[v] = f"Blood pressure field: {v}"

# ====================================================================
# 3. Create cleaned BP variables
# ====================================================================

df["SBP_mean"] = df[["BPXOSY1","BPXOSY2","BPXOSY3"]].mean(axis=1)
df["DBP_mean"] = df[["BPXODI1","BPXODI2","BPXODI3"]].mean(axis=1)
df["PP"] = df["SBP_mean"] - df["DBP_mean"]
df["MAP"] = df["DBP_mean"] + (df["PP"] / 3)

# Add descriptions
keep["SBP_mean"] = "Avg systolic BP (mmHg)"
keep["DBP_mean"] = "Avg diastolic BP (mmHg)"
keep["PP"] = "Pulse pressure"
keep["MAP"] = "Mean arterial pressure"

# ====================================================================
# 4. Drop all other columns
# ====================================================================

final_cols = list(keep.keys())
df2 = df[final_cols].copy()

# ====================================================================
# 5. Print summary of columns kept + descriptions
# ====================================================================

print("\nFINAL COLUMNS KEPT:")
for col, desc in keep.items():
    print(f" - {col}: {desc}")

print("\nTotal columns kept:", len(df2.columns))

# ====================================================================
# 6. Save cleaned dataset
# ====================================================================

df2.to_csv("NHANES_DIABETES_CLEAN.csv", index=False)
print("\nSaved → NHANES_DIABETES_CLEAN.csv")



FINAL COLUMNS KEPT:
 - SEQN: Participant ID
 - RIDAGEYR: Age (years)
 - RIAGENDR: Gender (1=Male,2=Female)
 - RIDRETH1: Race/Ethnicity
 - INDFMPIR: Income-to-poverty ratio
 - LBXGH: HbA1c (%)
 - LBDGLUSI: Fasting plasma glucose (mg/dL)
 - LBDLDL: LDL cholesterol (mg/dL)
 - URXUMA: Urine albumin (mg/L)
 - BMXBMI: BMI (kg/m2)
 - BMXWAIST: Waist circumference (cm)
 - DIQ010: Doctor told you have diabetes
 - DIQ050: Taking insulin now
 - BPXOSY1: Blood pressure field: BPXOSY1
 - BPXOSY2: Blood pressure field: BPXOSY2
 - BPXOSY3: Blood pressure field: BPXOSY3
 - BPXODI1: Blood pressure field: BPXODI1
 - BPXODI2: Blood pressure field: BPXODI2
 - BPXODI3: Blood pressure field: BPXODI3
 - BPXOPLS1: Blood pressure field: BPXOPLS1
 - BPXOPLS2: Blood pressure field: BPXOPLS2
 - BPXOPLS3: Blood pressure field: BPXOPLS3
 - BPAOARM: Blood pressure field: BPAOARM
 - BPAOCSZ: Blood pressure field: BPAOCSZ
 - SBP_mean: Avg systolic BP (mmHg)
 - DBP_mean: Avg diastolic BP (mmHg)
 - PP: Pulse pressure
 

In [4]:
import pandas as pd

# Load the cleaned BP-integrated dataset from your previous cell
df = pd.read_csv("NHANES_DIABETES_CLEAN.csv")

print("Before dropna:")
print("Rows:", df.shape[0], "Columns:", df.shape[1])

# ---- DROP ANY ROW THAT CONTAINS A NULL VALUE ----
df_dropna = df.dropna()

print("\nAfter dropna (any null removed):")
print("Rows:", df_dropna.shape[0], "Columns:", df_dropna.shape[1])

# Save the strict-clean version
df_dropna.to_csv("NHANES_BP_CLEAN_DROPNA.csv", index=False)

print("\nSaved as NHANES_BP_CLEAN_DROPNA.csv")


Before dropna:
Rows: 11933 Columns: 28

After dropna (any null removed):
Rows: 319 Columns: 28

Saved as NHANES_BP_CLEAN_DROPNA.csv


In [5]:
import pandas as pd

df = pd.read_csv("NHANES_DIABETES_CLEAN.csv")

# Key variables needed for prediction/label
critical_vars = [
    "DIQ010",
    "LBDGLUSI",
    "LBXGH",
    "BMXBMI",
    "SBP_mean",
    "DBP_mean"
]

# Drop rows missing any critical variable
df2 = df.dropna(subset=critical_vars)

print("Before:", df.shape)
print("After dropping only critical missing rows:", df2.shape)

# Median/mode impute the rest
for col in df2.columns:
    if df2[col].dtype in ["float64", "int64"]:
        df2[col] = df2[col].fillna(df2[col].median())
    else:
        df2[col] = df2[col].fillna(df2[col].mode()[0])

df2.to_csv("NHANES_CLEAN_METHOD2.csv", index=False)
print("Saved as NHANES_CLEAN_METHOD2.csv")


Before: (11933, 28)
After dropping only critical missing rows: (3522, 28)
Saved as NHANES_CLEAN_METHOD2.csv


C:\Users\lenovo\AppData\Local\Temp\ipykernel_3248\2183378303.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].fillna(df2[col].median())
C:\Users\lenovo\AppData\Local\Temp\ipykernel_3248\2183378303.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].fillna(df2[col].mode()[0])


In [6]:
import pandas as pd

df = pd.read_csv("NHANES_DIABETES_CLEAN.csv")

# Drop only missing DIQ010 (label)
df3 = df.dropna(subset=["DIQ010"])

# Impute all remaining
for col in df3.columns:
    if df3[col].dtype in ["float64", "int64"]:
        df3[col] = df3[col].fillna(df3[col].median())
    else:
        df3[col] = df3[col].fillna(df3[col].mode()[0])

df3.to_csv("NHANES_CLEAN_METHOD3.csv", index=False)
print("Saved as NHANES_CLEAN_METHOD3.csv")
print("Final shape:", df3.shape)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_3248\241369530.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[col] = df3[col].fillna(df3[col].median())
C:\Users\lenovo\AppData\Local\Temp\ipykernel_3248\241369530.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[col] = df3[col].fillna(df3[col].mode()[0])


Saved as NHANES_CLEAN_METHOD3.csv
Final shape: (11740, 28)
